In [ ]:
#!/usr/bin/env python
# pylint: disable=unused-argument
# This program is dedicated to the public domain under the CC0 license.

"""
Simple Bot to reply to Telegram messages.

First, a few handler functions are defined. Then, those functions are passed to
the Application and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.

Usage:
Basic Echobot example, repeats messages.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging

from telegram import ForceReply, Update
from telegram.ext import Application, CommandHandler, ContextTypes, MessageHandler, filters

# Enable logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
# set higher logging level for httpx to avoid all GET and POST requests being logged
logging.getLogger("httpx").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    await update.message.reply_html(
        rf"Hi {user.mention_html()}!",
        reply_markup=ForceReply(selective=True),
    )


async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /help is issued."""
    await update.message.reply_text("Help!")


async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Echo the user message."""
    await update.message.reply_text(update.message.text)


def main() -> None:
    """Start the bot."""
    # Create the Application and pass it your bot's token.
    application = Application.builder().token("7561599888:AAF0uFMJrNRovEZMJC_K09DPbUaQL7uWH5U").build()

    # on different commands - answer in Telegram
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, echo))

    # Run the bot until the user presses Ctrl-C
    application.run_polling(allowed_updates=Update.ALL_TYPES)


if __name__ == "__main__":
    main()

In [1]:
import os
os.chdir("/home/carnufex/butlerbot")
from src.chains import category_chain

note = category_chain.invoke({
    "input": "I have a meeting tomorrow at 10 am with the tax man",
    
})
note

Note(id=1, created='calendar', tags='personal', content='Meeting with the tax man at 10 am.', created_at=datetime.date(2025, 4, 16), date=datetime.date(2025, 4, 17))

In [2]:
## turn NOTE into a polars df with same columns as the one in the database
import polars as pl
note.model_dump_json()

'{"id":1,"created":"calendar","tags":"personal","content":"Meeting with the tax man at 10 am.","created_at":"2025-04-16","date":"2025-04-17"}'

In [3]:
notes = pl.DataFrame(note.model_dump())

In [4]:
notes.write_parquet("/home/carnufex/butlerbot/data/notes.parquet", compression="lz4")

In [5]:
notes

id,created,tags,content,created_at,date
i64,str,str,str,date,date
1,"""calendar""","""personal""","""Meeting with the tax man at 10…",2025-04-16,2025-04-17
